In [ ]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    
    import pandas as pd
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    import os
    import sys
    
    import os
    import numpy as np
    import tensorflow as tf
    
    from elasticsearch import helpers
    
    from gensim.models import Word2Vec, KeyedVectors
    
    import nltk
    import re
    from nltk.corpus import stopwords
    nltk.download('stopwords')
    
    import spacy

    print("Loaded  .. . . . . . . .")
except Exception as E:
    print("Some Modules are Missing {} ".format(E))

In [ ]:
df = pd.read_csv("val_knowledgebase.csv")
print("read")

In [ ]:
df.head()

In [ ]:
from bert_serving.client import BertClient
bc = BertClient()

In [ ]:
country=df["country"]
country_set=set(country)
product=df["product"]
product_set=set(product)
print("countries : "+str(country_set))
print("products : "+str(product_set))

In [ ]:
#load fasttext modelnlp=spacy.load('en_core_web_lg')
data_path_fast= "/home/srinivas/Downloads"
path_to_model_fasttext= os.path.join(data_path_fast,'wiki-news-300d-1M.vec')
print("loading fasttext model...")
fasttext_model = KeyedVectors.load_word2vec_format(path_to_model_fasttext)
print("done loading fasttext model...")

In [ ]:
#load spacy model
nlp=spacy.load('en_core_web_lg')
print("spacy model loaded....")

In [ ]:
#load w2v model
data_path_w2v= "/home/srinivas/Downloads"
path_to_model = os.path.join(data_path_w2v,'GoogleNews-vectors-negative300.bin')
print("loading...")
w2v_model = KeyedVectors.load_word2vec_format(path_to_model, binary=True)
print("done loading...")

In [ ]:
def clean(text):
    review = re.sub('[^a-zA-Z]', ' ',text )#replace everything excepts aphabets
    review = review.lower()
    review = review.split()
    
    review = [word for word in review if not word in stopwords.words('english')]
    return review

In [ ]:
#Dataset Preprocessing(removing stop words and html tags and punctuations)
def create_corpus():
    corpus = []
    for i in range(0, len(df)):
        review = re.sub('[^a-zA-Z]', ' ', df["query"][i])#replace everything excepts aphabets
        review = review.lower()
        review = review.split()

        review = [word for word in review if not word in stopwords.words('english')]
        corpus.append(review)
    print("done")
    return corpus

In [ ]:
def create_corpus_spacy():
    corpus = []
    for i in range(0, len(df)):
        review = re.sub('[^a-zA-Z]', ' ', df["atom"][i])#replace everything excepts aphabets
        review = review.lower()
        review = review.split()

        review = [word for word in review if not word in stopwords.words('english')]
        review=" ".join(review)
        corpus.append(review)
    print("done")
    return corpus

In [ ]:
def embedd_spacy():
    corpus=create_corpus_spacy()
    print(corpus)
    feats=[]
    for x in corpus:
        features=nlp(x)
        features=features.vector
        if np.count_nonzero(features)==0:
            return [np.ones(300)]
        feats.append(features)
    return feats

In [ ]:
def embedd_corpus_fasttext():
    corpus=create_corpus()
    print(corpus)
    feats=[]
    Dimension=300
    for sentence in corpus:
        feat_for_this=np.zeros(Dimension)
        count=0
        for word in sentence:
            if word in fasttext_model:
                feat_for_this += fasttext_model[word]
                count+=1
        if count==0:
            count=1
            print("yessss")
            feat_for_this=np.ones(Dimension)
        feats.append(feat_for_this/count)
    return feats

In [ ]:
def embedd_corpus_w2v():
    corpus=create_corpus()
    print(corpus)
    feats=[]
    Dimension=300
    for sentence in corpus:
        feat_for_this=np.zeros(Dimension)
        count=0
        for word in sentence:
            if word in w2v_model:
                feat_for_this += w2v_model[word]
                count+=1
        if count==0:
            count=1
            print("yessss")
            feat_for_this=np.ones(Dimension)
        feats.append(feat_for_this/count)
    return feats

In [ ]:
def single_spacy(que):
    que=clean(que)
    que=" ".join(que)
    print(que)
    features=nlp(que)
    features=features.vector
    if np.count_nonzero(features)==0:
        print("This is a UNIT VECTOR!!")
        return np.ones(300)
    return features

def single_w2v(que):
    que=clean(que)
    print(que)
    Dimension=300
    
    feat_for_this=np.zeros(Dimension)
    count=0
    for word in que:
        if word in w2v_model:
            feat_for_this += w2v_model[word]
            count+=1
    if count==0:
        print("This is a UNIT VECTOR!!")
        return np.ones(Dimension)
    #print(count)
    return (feat_for_this/count)

def single_fasttext(que):
    que=clean(que)
    print(que)
    Dimension=300
    
    feat_for_this=np.zeros(Dimension)
    count=0
    for word in que:
        if word in fasttext_model:
            #print(word)
            feat_for_this += fasttext_model[word]
            count+=1
    if count==0:
        print("This is a UNIT VECTOR!!")
        return np.ones(Dimension)
    #print(count)
    return (feat_for_this/count)



In [ ]:
fast_vec=embedd_corpus_fasttext()

In [ ]:
df["atom"][0]

In [ ]:
import pickle

In [ ]:
with open('/home/srinivas/Desktop/validation_arrays/fasttext_answer_.pkl','wb') as f:
     pickle.dump(fast_vec, f)

In [ ]:
with open('/home/srinivas/Desktop/validation_arrays/fasttext_answer_.pkl','rb') as f:
     x_answer = pickle.load(f)

In [ ]:
len(x_answer)

In [ ]:
fast_vec_question=embedd_corpus_fasttext()

In [ ]:
df["atom"][0]

In [ ]:
with open('/home/srinivas/Desktop/validation_arrays/fasttext_question.pkl','wb') as f:
     pickle.dump(fast_vec_question, f)

In [ ]:
with open('/home/srinivas/Desktop/validation_arrays/fasttext_question.pkl','rb') as f:
     x_ques = pickle.load(f)

In [ ]:
len(x_ques)

In [ ]:
w2v_vec_q=embedd_corpus_w2v()

In [ ]:
w2v_vec_q

In [ ]:
import pickle
with open('/home/srinivas/Desktop/validation_arrays/w2v_question.pkl','wb') as f:
     pickle.dump(w2v_vec_question, f)

In [ ]:
with open('/home/srinivas/Desktop/validation_arrays/w2v_question.pkl','rb') as f:
     w2v_ques = pickle.load(f)

In [ ]:
w2v_ques

In [ ]:
spacy_vec=embedd_spacy()

In [ ]:
bert_vec=bc.encode(list(df["atom"]))

In [ ]:
len(bert_vec[0])

In [ ]:
import pickle

In [ ]:
with open('/home/srinivas/Desktop/validation_arrays/bert/bert_answer.pkl','wb') as f:
     pickle.dump(bert_vec, f)

In [ ]:
import pickle
with open('/home/srinivas/Desktop/validation_arrays/bert/bert_answer.pkl','rb') as f:
     x77 = pickle.load(f)

In [ ]:
len(x77[0])

In [ ]:
bert_a=list(df["atom"])
bert_a

In [ ]:
len(bert_a)

In [ ]:
bert_vec_answer=bc.encode(bert_a)

In [ ]:
len(bert_vec_answer[0])

In [ ]:
import pickle
with open('/home/srinivas/Desktop/validation_arrays/bert/bert_answer.pkl','wb') as f:
     pickle.dump(bert_vec_answer, f)

In [ ]:
with open('/home/srinivas/Desktop/validation_arrays/bert/bert_answer.pkl','rb') as f:
     xa= pickle.load(f)

In [ ]:
len(xa[0])

In [ ]:
len(bert_vec_question[0])

In [ ]:
df["query"]

In [ ]:
print(len(spacy_vec))
print(len(fast_vec))
print(len(w2v_vec))

In [ ]:
print(len(spacy_vec[0]))
print(len(fast_vec[0]))
print(len(w2v_vec[0]))

In [ ]:
df["fasttext"]=fast_vec
df["w2v"]=w2v_vec
df["spacy"]=spacy_vec

In [ ]:
df.head()

In [ ]:
df.to_pickle("./validation_data_with_3_w2v_vectors.pkl")

In [ ]:
print(tf.__version__)

In [ ]:
Settings = {
   "settings":{
      "number_of_shards":1,
      "number_of_replicas":0
   },
   "mappings":{
       "dynamic": "true",
        "_source": {
        "enabled": "true"
        },
       "properties":{
            "answer": {
            "type": "text"
            },
          "fasttext_vector":{
         "type":"dense_vector",
         "dims":300
      },
        "w2v_vector":{
         "type":"dense_vector",
         "dims":300
      },
        "spacy_vector":{
         "type":"dense_vector",
         "dims":300
      },
          "country":{  
            "type":"text"
         },   
          "product":{  
            "type":"text"
         }
    }
   }
}

In [ ]:
ENDPOINT = "http://localhost:9200/"
es = Elasticsearch(timeout=600,hosts=ENDPOINT)
print("python client created")

In [ ]:
#creating the database
IndexName = 'ensemble_w2v_models'
my = es.indices.create(index=IndexName, ignore=[400,404], body=Settings)

In [ ]:
my

In [ ]:
df.columns

In [ ]:
df22 = df.to_dict('records')
df22

In [ ]:
len(df22)

In [ ]:
arr=[]
for c, line in enumerate(df22):
    arr.append({"answer":line.get("atom",""),
    'fasttext_vector':line.get("fasttext",""),
    'w2v_vector':line.get("w2v",""),
    'spacy_vector':line.get("spacy",""),
    'country':line.get("country","").lower().strip(),
    'product':line.get("product","").lower().strip()})

In [ ]:
arr

In [ ]:
for i in range(0,len(arr)):

    resp=es.index(index=IndexName,doc_type="_doc",id=i,body=arr[i])

In [ ]:
resp

In [ ]:
que="what is Crestor used for in canada?"
print(que)
temp_prod=df["product"]
temp_country=df["country"]
product_list=list(set(temp_prod))
print(product_list)
country_list=list(set(temp_country))
print(country_list)

In [ ]:
w2v_1=single_w2v(que)


In [ ]:
def list_generator():    
    temp_prod=df["product"]
    temp_country=df["country"]

    product_list=list(set(temp_prod))
    country_list=list(set(temp_country))

    for i in range(0,len(product_list)):
        product_list[i]=product_list[i].lower()

    for i in range(0,len(country_list)):
        country_list[i]=country_list[i].lower().strip()

    prods=[]

    for x in product_list:
        temp=x.split("(")
        for i in range(0,len(temp)):
            temp[i]=temp[i].replace(")","")
            temp[i]=temp[i].strip()
            prods.append(temp[i])
    return [prods,country_list]

    

In [ ]:
list_generator()

In [ ]:
def query_creator(que,lists):    
    prods=lists[0]
    country_list=lists[1]
    print(prods)
    print(country_list)
    
    spacy_vec=single_spacy(que)
    w2v_vec=single_w2v(que)
    fasttext_vec=single_fasttext(que)
    #print(que_vec)
    #print(que)
    que=que.lower()
    p=""
    c=""
    for x in prods:
        if x in que:
            p=x
            print(p)
            break
    for x in country_list:
        if x in que:
            c=x
            print(c)

    script_query={}
    if p!="" and c!="":
        print("1...........")
        script_query = {
        "script_score": {
        "query": {
    "bool": {
      "must": [
        {
          "match": {
            "product": p
          }
        }
      ],
      "should": [
        {
          "match": {
            "country": c
          }
        }
      ]
    }
  },
        "script": {
        "source": "((Math.abs(cosineSimilarity(params.spacy_vector, doc['spacy_vector'])))+(Math.abs(cosineSimilarity(params.w2v_vector, doc['w2v_vector'])))+(Math.abs(cosineSimilarity(params.fasttext_vector, doc['fasttext_vector']))))/3",
        "params": {"spacy_vector": spacy_vec,"w2v_vector": w2v_vec,"fasttext_vector": fasttext_vec}
        }}}

    elif p!="" and c=="":
        print("2...........")
        script_query = {
        "script_score": {
        "query": {"match": {
              "product": {
                "query":p
              }}},
        "script": {
        "source": "((Math.abs(cosineSimilarity(params.spacy_vector, doc['spacy_vector'])))+(Math.abs(cosineSimilarity(params.w2v_vector, doc['w2v_vector'])))+(Math.abs(cosineSimilarity(params.fasttext_vector, doc['fasttext_vector']))))/3",
        "params": {"spacy_vector": spacy_vec,"w2v_vector": w2v_vec,"fasttext_vector": fasttext_vec}
        }}}
    elif p=="" and c!="":
        print("3...........")
        script_query = {
        "script_score": {
        "query": {"match": {
              "country": {
                "query":c
              }}},
        "script": {
        "source": "((Math.abs(cosineSimilarity(params.spacy_vector, doc['spacy_vector'])))+(Math.abs(cosineSimilarity(params.w2v_vector, doc['w2v_vector'])))+(Math.abs(cosineSimilarity(params.fasttext_vector, doc['fasttext_vector']))))/3",
        "params": {"spacy_vector": spacy_vec,"w2v_vector": w2v_vec,"fasttext_vector": fasttext_vec}
        }}}
    else:
        print("4...........")
        script_query = {
        "script_score": {
        "query": {"match_all": {}},
        "script": {
        "source": "((Math.abs(cosineSimilarity(params.spacy_vector, doc['spacy_vector'])))+(Math.abs(cosineSimilarity(params.w2v_vector, doc['w2v_vector'])))+(Math.abs(cosineSimilarity(params.fasttext_vector, doc['fasttext_vector']))))/3",
        "params": {"spacy_vector": spacy_vec,"w2v_vector": w2v_vec,"fasttext_vector": fasttext_vec}
        }}}
    return  script_query

In [ ]:
lists=list_generator()

In [ ]:
lists

In [ ]:
que="what is Crestor used for ?"

In [ ]:
script_query=query_creator(que,lists)
script_query

In [ ]:
ans=es.search(index=IndexName,body={"size": 10,"query": script_query,"_source": {"includes": ["answer","country","product"]}})

In [ ]:
ans

In [ ]:
def questions():
    count=0
    que_arr=[]
    for x in df["query"]:
        sent=x.split("\n")
        for q in sent:
            if re.search('[a-zA-Z]',q)!=None:
                que_arr.append([q,count])
        count+=1
    return que_arr

In [ ]:
que_arr=questions()

In [ ]:
len(que_arr)

In [ ]:
que_arr

In [ ]:

def rank(ans):
    required=ans['hits']
    main_arr=required["hits"]
    count=1
    ids__=dict()
    for x in main_arr:
        id_no= x["_id"]

        score=x["_score"]

        answer_=x["_source"]["answer"]
        
        ids__[id_no.strip()]=1/count
        count+=1
    return ids__
    

In [ ]:
ranker=rank(ans)

In [ ]:
ranker

In [ ]:
ans=es.search(index=IndexName,body={"size": 10,"query": script_query,"_source": {"includes": ["answer"]}})

In [ ]:
def mrr():
    total=0
    count=0
    recall_count=0
    lists_all=list_generator()
    print(lists_all)
    for x in que_arr:
        question_=x[0].strip()
        print(question_)
        id_number=str(x[1])

        script_query_curr=query_creator(question_,lists_all)
        ans=es.search(index=IndexName,body={"size": 10,"query": script_query_curr,"_source": {"includes": ["answer","product","country"]}})
        ranker=rank(ans)
        if id_number in ranker:
            total+=ranker[id_number]
            print(ranker[id_number])
            recall_count+=1
        else:
            print(0)
        count+=1

    print("mrr is"+str(total/count))
    print("recall is"+str(recall_count/count))
    return
        

In [ ]:
mrr()